In [ ]:
# import libraries
from asyncio import sleep
from matipo import SEQUENCE_DIR, GLOBALS_DIR
from matipo.sequence import Sequence
from matipo.util.autophase import autophase
from matipo.util.decimation import decimate
from matipo.util.fft import fft_reconstruction
from pathlib import Path
import ipyvolume as ipv
from matplotlib import cm
import numpy as np
import matplotlib.pyplot as plt

# progress_handler for Sequence.run() that simply prints the progress
def print_progress(p, l):
    if p%(l//4)==0: # only print 4 times per run
        print(p, '/', l)
# set save directory and base file name
SAVE_DIR = '/home/data/flow-2D-SE'
SAVE_NAME = 'example'

# make the save directory if it doesn't exist
Path(SAVE_DIR).mkdir(parents=True, exist_ok=True)

In [ ]:
# load pulse sequence
seq = Sequence(SEQUENCE_DIR+'flow-SE.py')

# load relevant global variables
seq.loadpar(GLOBALS_DIR+'hardpulse_90.yaml')
seq.loadpar(GLOBALS_DIR+'hardpulse_180.yaml')
seq.loadpar(GLOBALS_DIR+'frequency.yaml')
seq.loadpar(GLOBALS_DIR+'shims.yaml')

DECIMATION = 4

G_CAL = 8.0e6  # Hz/m at max grad
G_FLOW = np.array((0, 0, 0.3))  # keep same amplitude

# --- Adjust t_flow and t_flow_spacing for VENC ~0.700 m/s ---
t_flow = 0.000385758  # seconds, tuned for 0.700 m/s
t_flow_spacing = t_flow

# set parameters for a 32x32 X/Y 2D projection SE image
seq.setpar(
    # duration of phase pulse, not including ramping
    t_phase=240e-6,

    # phase encode Y axis
    n_phase_1=32,
    g_phase_1=(0, -0.8, 0),

    # frequency encode X axis
    g_phase_read=(-0.8, 0, 0),
    t_dw=10e-6,
    n_samples=DECIMATION*32,
    g_read=(0.3, 0, 0),

    g_spoil=(0.2, 0, 0),  # same direction as read gradient
    t_spoil=1000e-6,

    t_flow=t_flow,
    t_flow_spacing=t_flow_spacing,
    g_flow=(0, 0, 0),a
    g_flow_tweak=(0.998, 0.998, 0.998),

    t_echo=4e-3,
    t_end=200e-3,
    n_scans=4
)

# calculate VENC for reference
VENC = np.pi / (2 * np.pi * G_CAL * 2 * np.linalg.norm(G_FLOW) * seq.par.t_flow * seq.par.t_flow_spacing)
print(seq.par)
print('Venc (m/s):', VENC)

# estimate total scan time
time_est_s = 2*(seq.par.n_scans*seq.par.n_phase_1*seq.par.n_phase_2*(seq.par.t_end + seq.par.t_echo + seq.par.t_spoil))
print('total time (mins):', time_est_s/60)

# Acquire negative flow
print('Acquiring image with negative flow encoding')
seq.setpar(g_flow=(0, 0, -G_FLOW[2]))
await seq.run(progress_handler=print_progress)
y_vzn = decimate(np.reshape(seq.data, (seq.par.n_phase_1, seq.par.n_samples)), DECIMATION, axis=1)
np.save(f'{SAVE_DIR}/{SAVE_NAME}_venc0_700_vzn', y_vzn)
seq.savepar(f'{SAVE_DIR}/{SAVE_NAME}_venc0_700_vzn.yaml')

# Acquire positive flow
print('Acquiring image with positive flow encoding')
seq.setpar(g_flow=(0, 0, G_FLOW[2]))
await seq.run(progress_handler=print_progress)
y_vzp = decimate(np.reshape(seq.data, (seq.par.n_phase_1, seq.par.n_samples)), DECIMATION, axis=1)
np.save(f'{SAVE_DIR}/{SAVE_NAME}_venc0_700_vzp', y_vzp)
seq.savepar(f'{SAVE_DIR}/{SAVE_NAME}_venc0_700_vzp.yaml')

# run FFT with upscaling
im_vzn = fft_reconstruction(y_vzn, upscale_factor=2, gaussian_blur=0.5)
im_vzp = fft_reconstruction(y_vzp, upscale_factor=2, gaussian_blur=0.5)

In [ ]:
filt = np.abs(im_vzn)*np.abs(im_vzp)
thresh = 0.25*np.max(filt)
filt[filt > thresh] = 1
filt[filt <= thresh] = 0
cmap = 'gray'
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15,5))
axes[0].imshow(np.abs(im_vzn), cmap=cmap)
axes[1].imshow(np.abs(im_vzp), cmap=cmap)
axes[2].imshow(filt, cmap=cmap)
plt.show()
# plot the phase of the images
cmap='hsv' 
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10,5))
axes[0].imshow(filt*np.angle(im_vzn), vmin=-np.pi, vmax=np.pi, cmap=cmap)
axes[1].imshow(filt*np.angle(im_vzp), vmin=-np.pi, vmax=np.pi, cmap=cmap)
plt.show()

In [ ]:
# calculate velocity from phase difference (uncalibrated, arbitrary units)
vz = filt*np.angle(im_vzp/im_vzn) # dividing complex numbers will take the difference of their angles
vz_mag_max = np.nanmax(np.sqrt(vz*vz))
# plot velocity profile
plt.figure(figsize=(10,5))
plt.imshow(vz, cmap='coolwarm', vmin=-vz_mag_max, vmax=vz_mag_max)
plt.colorbar()
plt.show()

In [ ]:
colormap = cm.coolwarm
Z = vz
znorm = Z - np.nanmin(Z)
znorm /= (np.nanmax(znorm)-np.nanmin(znorm))
color = colormap(znorm)
X, Y = np.meshgrid(range(Z.shape[0]),range(Z.shape[1]))
ipv.figure()
ipv.style.axes_off()
ipv.style.box_off()
ipv.plot_surface(X, Y, Z, color=color)
ipv.show()
import numpy as np
import matplotlib.pyplot as plt
# im_vzp = complex image with positive flow encoding
# im_vzn = complex image with negative flow encoding
# 1. Compute phase difference image (wrapped between -π and π)
phase_diff = np.angle(im_vzp / im_vzn)
# 2. Calculate velocity map (units same as VENC, usually m/s)
velocity_map = (phase_diff / np.pi) * VENC
magnitude = np.abs(im_vzp) * np.abs(im_vzn)
mask = magnitude > 0.2 * np.max(magnitude)  # adjust threshold as needed
velocity_map_masked = np.where(mask, velocity_map, np.nan)
plt.figure(figsize=(8,6))
plt.imshow(velocity_map_masked, cmap='coolwarm', vmin=-VENC, vmax=VENC)
plt.colorbar(label='Velocity (m/s)')
plt.title('Velocity map calculated from phase difference')
plt.show()

mean_velocity = np.nanmean(velocity_map_masked)
max_velocity = np.nanmax(np.abs(velocity_map_masked))
median_velocity = np.nanmedian(velocity_map_masked)

print(f"Mean velocity (m/s): {mean_velocity:.4f}")
print(f"Median velocity (m/s): {median_velocity:.4f}")
print(f"Max velocity magnitude (m/s): {max_velocity:.4f}")
